In [ ]:
dataset_name = "W19_comb"
df_list = [ "BES_Panel", "BES_reduced_with_na"]

In [ ]:
5+5

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

%matplotlib inline

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re

sns.set();
sns.set_palette("colorblind")

from IPython.display import display, display_html, HTML
from IPython.core.debugger import set_trace
# plt.rcParams["axes.grid"] = False

import Jupyter_module_loader
from utility import *
import gaussian_kde

import warnings
warnings.filterwarnings('ignore')

import holoviews as hv
from holoviews import opts

encoding = "ISO-8859-1"

# you should clone this git to a subdirectory called 'BES_analysis_code' (in some directory - I call it BES_analysis - doesn't matter though)
# %matplotlib inline
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

for df in df_list:
    if df=="BES_Panel":
        globals()[df]  = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
    else:
        globals()[df]  = pd.read_pickle(data_subfolder + df + '.zip',compression='zip' )
        globals()[df].replace(-1,np.nan,inplace=True)
  
(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)

# get full set of inferred "cross wave" auth-lib/left-right values and ages
pan_dataset_allr_values = pd.read_csv(BES_small_data_files + "pan_dataset_allr_values"+".csv")
pan_dataset_ages = pd.read_pickle(BES_small_data_files + "pan_dataset_ages"+".zip", compression='zip')

In [ ]:
BES_Panel.loc[30929, 35852, 38388, 39244, 76846, 77218, 77875, 89977]

In [426]:
# ALERT ALERT AWOOOGA - Heinz Brandenburg
# https://twitter.com/HzBrandenburg/status/1258414163833094154

# (1) check this for specific wave7+/wave6+ respondent split
# (2) think about comparing datasets with datasets on matching variables!

In [11]:
respondent_in_first_six_waves = BES_Panel[search(BES_Panel,"wave\d+").index[0:6]].any(axis=1)
respondent_after_first_six_waves = ~BES_Panel[search(BES_Panel,"wave\d+").index[0:6]].any(axis=1)


In [24]:
vote2010 = list(search(BES_reduced_with_na,"profile_past_vote_2010").index)
vote2015 = list(search(BES_reduced_with_na,"profile_past_vote_2015").index)

cor_mat = BES_reduced_with_na[vote2010+vote2015][respondent_after_first_six_waves].astype('float32').corr().loc[vote2010,vote2015]
cor_mat.columns = [x.replace("profile_past_vote_","") for x in cor_mat.columns]
cor_mat.index = [x.replace("profile_past_vote_","") for x in cor_mat.index]
cor_mat.round(2)
# very tempting to shorten variable names here!

,2015_Conservative,2015_Labour,2015_United Kingdom Independence Party (UKIP),2015_Liberal Democrat,2015_Green Party,2015_Scottish National Party (SNP),2015_Plaid Cymru,2015_British National Party (BNP)
2010_Conservative,-0.09,0.04,-0.01,-0.01,0.09,0.01,0.01,-0.00
2010_Labour,0.60,-0.40,-0.01,-0.15,-0.15,-0.10,-0.03,-0.01
2010_Liberal Democrat,-0.31,0.50,-0.10,-0.11,-0.06,-0.01,-0.02,-0.01
2010_Scottish National Party (SNP),-0.20,0.01,-0.05,0.34,0.08,-0.03,-0.02,-0.01
2010_United Kingdom Independence Party (UKIP),-0.03,-0.02,-0.01,-0.01,0.02,-0.01,0.46,-0.00
2010_Brexit Party,-0.01,-0.00,0.03,-0.01,-0.01,0.00,0.00,0.00
2010_British National Party (BNP),-0.08,-0.03,-0.02,-0.02,0.28,-0.01,0.00,-0.01
2010_Plaid Cymru,-0.08,-0.07,-0.04,-0.03,-0.03,0.59,-0.01,-0.00
2010_10.0,-0.08,-0.10,0.32,-0.04,-0.03,-0.02,-0.01,0.04
2010_Green Party,-0.04,-0.04,0.14,-0.02,-0.02,-0.01,-0.01,0.11


In [25]:
vote2010 = list(search(BES_reduced_with_na,"profile_past_vote_2010").index)
vote2015 = list(search(BES_reduced_with_na,"profile_past_vote_2015").index)

cor_mat = BES_reduced_with_na[vote2010+vote2015][respondent_in_first_six_waves].astype('float32').corr().loc[vote2010,vote2015]
cor_mat.columns = [x.replace("profile_past_vote_","") for x in cor_mat.columns]
cor_mat.index = [x.replace("profile_past_vote_","") for x in cor_mat.index]
cor_mat.name = "corr for respondent_in_first_six_waves"
cor_mat.round(2)
# very tempting to shorten variable names here!

,2015_Conservative,2015_Labour,2015_United Kingdom Independence Party (UKIP),2015_Liberal Democrat,2015_Green Party,2015_Scottish National Party (SNP),2015_Plaid Cymru,2015_British National Party (BNP)
2010_Conservative,0.63,-0.40,0.05,-0.12,-0.10,-0.20,-0.07,-0.02
2010_Labour,-0.34,0.55,-0.12,-0.13,-0.05,-0.04,-0.03,0.01
2010_Liberal Democrat,-0.21,0.00,-0.05,0.33,0.15,-0.05,-0.00,0.01
2010_Scottish National Party (SNP),-0.15,-0.14,-0.08,-0.06,-0.04,0.65,-0.02,-0.01
2010_United Kingdom Independence Party (UKIP),-0.06,-0.09,0.32,-0.04,-0.02,-0.05,-0.01,-0.00
2010_Brexit Party,-0.01,0.00,-0.00,-0.01,0.02,0.00,-0.00,-0.00
2010_British National Party (BNP),-0.04,-0.05,0.18,-0.03,-0.01,-0.02,-0.01,0.04
2010_Plaid Cymru,-0.05,-0.04,-0.01,-0.01,-0.01,-0.03,0.53,-0.00
2010_10.0,-0.00,-0.00,0.02,-0.00,-0.00,-0.00,-0.00,-0.00
2010_Green Party,-0.06,-0.01,-0.02,-0.01,0.20,-0.01,-0.00,-0.00


In [41]:
# respondent joined in wave
first_wave_joined_panel = BES_Panel[search(BES_Panel,"wave\d+").index].replace(False,np.nan).fillna(method='ffill',axis=1).replace(np.nan,False).astype('bool')
first_wave_joined_panel.columns = [int(x.replace("wave","")) for x in first_wave_joined_panel.columns]
# respondent joined in wave
joined_panel_before_wave = BES_Panel[search(BES_Panel,"wave\d+").index].replace(False,np.nan).fillna(method='bfill',axis=1).replace(np.nan,False).astype('bool')
joined_panel_before_wave.columns = [int(x.replace("wave","")) for x in joined_panel_before_wave.columns]

In [50]:
BES_Panel["profile_past_vote_2005"]

0                          Plaid Cymru
1                               Labour
2                         Conservative
3                          Plaid Cymru
4                                  NaN
                     ...              
97153                           Labour
97154                     Conservative
97155                     Conservative
97156                 I would not vote
97157    Scottish National Party (SNP)
Name: profile_past_vote_2005, Length: 97158, dtype: category
Categories (15, object): [I would not vote < Conservative < Labour < Liberal Democrat ... Change UK- The Independent Group < Brexit Party < An independent candidate < Don't know]

In [45]:
search(BES_Panel,"profile_past_vote_2010")

profile_past_vote_2010    65581
dtype: int64

In [51]:
Lab2010 = BES_reduced_with_na["profile_past_vote_2010_Labour"].astype('float32')
Con2015 = BES_reduced_with_na["profile_past_vote_2015_Conservative"].astype('float32')
Lab2010.corr(Con2015)
DNV2010 = BES_Panel["profile_past_vote_2010"]=="I would not vote"
DNV2005 = BES_Panel["profile_past_vote_2005"]=="I would not vote"

In [56]:
for wave in range(1,20):
    print( wave,
          DNV2010[first_wave_joined_panel[wave]].sum(),
          DNV2010[~first_wave_joined_panel[wave]].sum(),
          DNV2005[first_wave_joined_panel[wave]].sum(),
          DNV2005[~first_wave_joined_panel[wave]].sum(),
         )

1 3276 1725 3530 1261
2 3711 1290 3910 881
3 4220 781 4101 690
4 5000 1 4790 1
5 5001 0 4791 0
6 5001 0 4791 0
7 5001 0 4791 0
8 5001 0 4791 0
9 5001 0 4791 0
10 5001 0 4791 0
11 5001 0 4791 0
12 5001 0 4791 0
13 5001 0 4791 0
14 5001 0 4791 0
15 5001 0 4791 0
16 5001 0 4791 0
17 5001 0 4791 0
18 5001 0 4791 0
19 5001 0 4791 0


In [53]:
for wave in range(1,20):
    print( wave,
          DNV2005[first_wave_joined_panel[wave]].sum(),
          DNV2005[~first_wave_joined_panel[wave]].sum(),)

1 3530 1261
2 3910 881
3 4101 690
4 4790 1
5 4791 0
6 4791 0
7 4791 0
8 4791 0
9 4791 0
10 4791 0
11 4791 0
12 4791 0
13 4791 0
14 4791 0
15 4791 0
16 4791 0
17 4791 0
18 4791 0
19 4791 0


In [37]:
for wave in range(1,20):
    print( wave,
          Lab2010[first_wave_joined_panel[wave]].corr(Con2015[first_wave_joined_panel[wave]]),
          Lab2010[~first_wave_joined_panel[wave]].corr(Con2015[~first_wave_joined_panel[wave]]),)

1 -0.33900350011179525 0.3790529817722261
2 -0.33354399713657945 0.42420905669017617
3 -0.33014477037274087 0.46270106702963504
4 -0.33587673044276234 0.6011021127411934
5 -0.3356347655116438 0.6010523351723633
6 -0.33564976778694866 0.6011991301006138
7 -0.16022180295231347 0.6023572016452368
8 -0.033737447305364404 0.5900475869576092
9 -0.033737447305364404 0.5900475869576092
10 0.02483885165616949 0.5891231321637798
11 0.04509423777486178 0.592166095040169
12 0.060027628430214156 0.5962549678185115
13 0.060027628430214156 0.5962549678185115
14 0.08502725309580596 nan
15 0.08502725309580596 nan
16 0.08502725309580596 nan
17 0.08502725309580596 nan
18 0.08502725309580596 nan
19 0.08502725309580596 nan


In [42]:
for wave in range(1,20):
    print( wave,
          Lab2010[joined_panel_before_wave[wave]].corr(Con2015[joined_panel_before_wave[wave]]),
          Lab2010[~joined_panel_before_wave[wave]].corr(Con2015[~joined_panel_before_wave[wave]]),)

1 0.08502725309580596 nan
2 0.08502725309580596 nan
3 0.08502725309580596 nan
4 0.08502725309580596 nan
5 0.08502725309580596 nan
6 0.08502725309580596 nan
7 0.11081420536264824 -0.32235313060698334
8 0.1151875956557256 -0.3158463977615409
9 0.114015058161784 -0.18792851912508743
10 0.11352194532306659 -0.10093937631090157
11 0.11564114733797397 -0.08253677077578192
12 0.1167153036114765 -0.07555602857812743
13 0.11560886904191217 -0.03486932713532534
14 0.1105933739777556 0.016917860458299037
15 0.10414216435039043 0.04601540549925962
16 0.10436390321821368 0.05148351066147123
17 0.0998090536335408 0.06359750804752981
18 0.09743845928014834 0.06924687072194988
19 0.11185057454786203 0.06668825450339508


In [38]:
for wave in range(1,20):
    print( wave,
          Lab2010[ BES_Panel["wave"+str(wave)] ].corr(Con2015[ BES_Panel["wave"+str(wave)] ]),
          Lab2010[ ~BES_Panel["wave"+str(wave)] ].corr(Con2015[ ~BES_Panel["wave"+str(wave)] ]),)

1 -0.33900350011179525 0.3790529817722261
2 -0.334137714115336 0.39431679603445763
3 -0.32920697908113544 0.3798416844239813
4 -0.34033270628967366 0.4957740874875053
5 -0.3374497189886388 0.5000117627074612
6 -0.33510495580765304 0.5074886949520409
7 -0.10712961254285307 0.28554286114009114
8 0.04409540471844286 0.1467064901896246
9 0.02190133567900101 0.1581251257059908
10 0.09484202942070685 0.07398177543898617
11 0.09928769098451251 0.06839478415727204
12 0.11031763339065578 0.04667384165336662
13 0.10582693010172996 0.05907182236132919
14 0.1355699056005412 0.030861716728359204
15 0.1312333809091834 0.04666021847530593
16 0.13804351541781557 0.05438931406115467
17 0.11629764195482728 0.06507472589336527
18 0.10067480610773795 0.07048369586635639
19 0.11185057454786203 0.06668825450339508


In [13]:
BES_Panel["profile_past_vote_2010"][respondent_after_first_six_waves]

2                         Conservative
4                         Conservative
5                                  NaN
7                                  NaN
9        Scottish National Party (SNP)
                     ...              
97148                     Conservative
97151                              NaN
97152                              NaN
97155                     Conservative
97157    Scottish National Party (SNP)
Name: profile_past_vote_2010, Length: 54161, dtype: category
Categories (14, object): [I would not vote < Conservative < Labour < Liberal Democrat ... 10 < Change UK- The Independent Group < Brexit Party < Don't know]

In [ ]:
## past voting behaviour


# profile_past_vote_2005
    # date relative to W1 - February 2014 - March 2014
    # 5 May 2005
    # -8.818617385352498 yrs
# profile_past_vote_2010
    # date relative to W1 - February 2014 - March 2014
    # 6 May 2010
    # -3.8165639972621492

In [8]:
td = (pd.to_datetime("5 May 2005") - pd.to_datetime("28 Feb 2014") )
td.days/365.25

In [10]:
td = (pd.to_datetime("6 May 2010") - pd.to_datetime("28 Feb 2014") )
td.days/365.25

-8.818617385352498

In [278]:
td = (pd.to_datetime("7 May 2015") - pd.to_datetime("28 Feb 2014") )
td.days/365.25

1.1854893908281998

In [51]:
BES_Panel["profile_past_vote_2005"].value_counts().sum()

56631

In [58]:
BES_Panel["profile_past_vote_2010"].value_counts().sum()

65581

In [59]:
BES_Panel["profile_past_vote_2010"]

0                    British National Party (BNP)
1                                          Labour
2                                    Conservative
3                                     Plaid Cymru
4                                    Conservative
                           ...                   
97153                                      Labour
97154    United Kingdom Independence Party (UKIP)
97155                                Conservative
97156                            I would not vote
97157               Scottish National Party (SNP)
Name: profile_past_vote_2010, Length: 97158, dtype: category
Categories (14, object): [I would not vote < Conservative < Labour < Liberal Democrat ... 10 < Change UK- The Independent Group < Brexit Party < Don't know]

In [56]:
((pan_dataset_ages[ BES_Panel["profile_past_vote_2005"].isnull() ]["Age_at_W1"]-8.818617385352498)<=19).value_counts()

False    25471
True     15056
Name: Age_at_W1, dtype: int64

In [57]:
((pan_dataset_ages[ BES_Panel["profile_past_vote_2010"].isnull() ]["Age_at_W1"]-3.8165639972621492)<=19).value_counts()

False    23663
True      7914
Name: Age_at_W1, dtype: int64

In [15]:
search(BES_Panel,"ukCitizen")
# ukCitizen, euCitizen, commonwealthCitizen, otherCitizen

# ... misses "Irish" citizens who have access

ukCitizenW1     30575
ukCitizenW8      6338
ukCitizenW14    31063
ukCitizenW15    30842
ukCitizenW16    37959
ukCitizenW17     3564
dtype: int64

In [32]:
commonwealthCitizen = BES_Panel[search(BES_Panel,"commonwealthCitizen").index].fillna(method='ffill',axis=1)[search(BES_Panel,"commonwealthCitizen").index[-1]]

In [16]:
ukCitizen = BES_Panel[search(BES_Panel,"ukCitizen").index].fillna(method='ffill',axis=1)[search(BES_Panel,"ukCitizen").index[-1]]

In [31]:
(ukCitizen=="Yes").sum(),ukCitizen.notnull().sum()

(77413, 79532)

In [33]:
(commonwealthCitizen=="Yes").sum(),commonwealthCitizen.notnull().sum()

(2776, 79532)

In [35]:
((ukCitizen=="Yes")|(commonwealthCitizen=="Yes")).sum()

77836

In [36]:
79532-77836

1696

In [ ]:
ukCitizen = BES_Panel[search(BES_Panel,"registeredW\d+").index].fillna(method='ffill',axis=1)[search(BES_Panel,"registeredW\d+").index[-1]]

In [25]:
search(BES_Panel,"registeredW\d+")

registeredW2     25814
registeredW3     27819
registeredW4     31532
registeredW6     30258
registeredW7     30895
registeredW8     33502
registeredW11    30956
registeredW13    31136
registeredW17    34366
registeredW19    32177
dtype: int64

In [37]:
(BES_Panel[search(BES_Panel,"registeredW\d+").index]=="Don't know").sum()

registeredW2     581
registeredW3     684
registeredW4     644
registeredW6     323
registeredW7     503
registeredW8     416
registeredW11    473
registeredW13    279
registeredW17    588
registeredW19    259
dtype: int64

In [28]:
(BES_Panel[search(BES_Panel,"registeredW\d+").index]=="No").sum()

registeredW2     396
registeredW3     360
registeredW4     406
registeredW6     333
registeredW7     249
registeredW8     278
registeredW11    326
registeredW13    219
registeredW17    260
registeredW19    182
dtype: int64

In [23]:
(BES_Panel[search(BES_Panel,"reasonNotRegistered").index]=="Yes").sum()

reasonNotRegistered_1W2        46
reasonNotRegistered_2W2        32
reasonNotRegistered_3W2        25
reasonNotRegistered_4W2        48
reasonNotRegistered_5W2        30
reasonNotRegistered_6W2        40
reasonNotRegistered_7W2       109
reasonNotRegistered_8W2        26
reasonNotRegistered_noneW2      0
reasonNotRegistered_dkW2       32
reasonNotRegistered_1W3        51
reasonNotRegistered_2W3         9
reasonNotRegistered_3W3        29
reasonNotRegistered_4W3        43
reasonNotRegistered_5W3        39
reasonNotRegistered_6W3        24
reasonNotRegistered_7W3       116
reasonNotRegistered_8W3        22
reasonNotRegistered_noneW3      0
reasonNotRegistered_dkW3       26
reasonNotRegistered_1W4       119
reasonNotRegistered_2W4        12
reasonNotRegistered_3W4        26
reasonNotRegistered_4W4        47
reasonNotRegistered_5W4        48
reasonNotRegistered_6W4        29
reasonNotRegistered_7W4        77
reasonNotRegistered_8W4        31
reasonNotRegistered_noneW4      0
reasonNotRegis

In [44]:
df

,turnoutUKGeneralW1__Very likely that I will vote,turnoutUKGeneralW2__Very likely that I will vote,turnoutUKGeneralW3__Very likely that I will vote,turnoutUKGeneralW4__Very likely that I will vote,turnoutUKGeneralW5__Very likely that I will vote,turnoutUKGeneralW7__Very likely that I will vote,turnoutUKGeneralW8__Very likely that I will vote,turnoutUKGeneralW9__Very likely that I will vote,turnoutUKGeneralW10__Very likely that I will vote,turnoutUKGeneralW11__Very likely that I will vote,...,conPriorities2_schoolW4_Yes,conPriorities2_nhsW4_Yes,conPriorities2_costLiveW4_Yes,devoResponsibleWalesJusticeW2_The UK Parliament,conPriorities2_immigW4_Yes,conPriorities2_crimeW4_Yes,sharedContentOnline_1W5_Yes,partySiteOthW5_Yes,euCitizenW1_Yes,conPriorities2_natsecW4_Yes
1607,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1644,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1709,NaN,NaN,NaN,3.0,3.0,2.0,4.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2063,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN
2072,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96062,NaN,NaN,4.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN
96098,NaN,NaN,4.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
96186,NaN,NaN,NaN,4.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96681,4.0,3.0,4.0,3.0,4.0,4.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.0,0.0,NaN


In [46]:
%%time
pattern = "reasonNotRegistered_1W6_Yes"
name = "reasonNotRegistered_1W6_Yes"
corr_type = "spearman"
mask = BES_Panel["wave6"] & (BES_Panel["registeredW6"]=="No")
df = BES_reduced_with_na[mask].astype('float32')
corr_df,corr_type = make_corr_summary(df, name, corr_type, pattern=pattern,min_sample_size=10)
display_corr(corr_df, name, corr_type, text_wrap_length=50)

,r,p,N
partyMemberPastW10_Plaid Cymru,1.000000,0.00,10
freeSpeechMilitaryTeachW15__Definitely,0.990000,0.00,13
reasonNotRegistered_1W8_Yes,0.950000,0.00,36
noDealEmotions_5W15_Yes,0.820000,0.00,13
euID5W7__Strongly agree,0.760000,0.01,10
labHopeW14_Yes,0.760000,0.01,10
profile_work_statW14_Full time student,0.740000,0.00,63
warmMuslimW11__Very warm,0.730000,0.00,13
freeSpeechIslamistSpeechW15__Definitely,0.720000,0.01,13
workingStatusW14W15_Full time university student,0.720000,0.00,71


Wall time: 50.5 s


In [47]:
search(BES_Panel,"countryOfBirth")

countryOfBirth       36842
countryOfBirthW17    34366
dtype: int64

In [49]:
BES_Panel["countryOfBirth"].value_counts()

England                                 25332
Scotland                                 6011
Wales                                    2834
Other: European Union member country      704
Other: Rest of world                      703
Other: Commonwealth member country        601
Prefer not to answer                      272
Republic of Ireland                       199
Northern Ireland                          186
Name: countryOfBirth, dtype: int64

In [40]:
search(BES_reduced_with_na, "reasonNotRegistered_1W6")
# BES_Panel["reasonNotRegistered_1W6"]

reasonNotRegistered_1W6_Yes    260
dtype: int64

In [42]:
df

,turnoutUKGeneralW1__Very likely that I will vote,turnoutUKGeneralW2__Very likely that I will vote,turnoutUKGeneralW3__Very likely that I will vote,turnoutUKGeneralW4__Very likely that I will vote,turnoutUKGeneralW5__Very likely that I will vote,turnoutUKGeneralW7__Very likely that I will vote,turnoutUKGeneralW8__Very likely that I will vote,turnoutUKGeneralW9__Very likely that I will vote,turnoutUKGeneralW10__Very likely that I will vote,turnoutUKGeneralW11__Very likely that I will vote,...,conPriorities2_schoolW4_Yes,conPriorities2_nhsW4_Yes,conPriorities2_costLiveW4_Yes,devoResponsibleWalesJusticeW2_The UK Parliament,conPriorities2_immigW4_Yes,conPriorities2_crimeW4_Yes,sharedContentOnline_1W5_Yes,partySiteOthW5_Yes,euCitizenW1_Yes,conPriorities2_natsecW4_Yes
1607,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1644,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1709,NaN,NaN,NaN,3.0,3.0,2.0,4.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2063,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN
2072,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96062,NaN,NaN,4.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN
96098,NaN,NaN,4.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
96186,NaN,NaN,NaN,4.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96681,4.0,3.0,4.0,3.0,4.0,4.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.0,0.0,NaN


In [484]:
pan_dataset_votes = pd.DataFrame()

age_cut_off = 18

Probably_didnt_vote_list = ["Don't know","No, did not vote","No, I did not vote",2.0,99.0]
nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])
def add_cat_and_modify_df(pan_dataset_votes, col, mask, new_cat ):
    if new_cat not in pan_dataset_votes[col].cat.categories:
        pan_dataset_votes[col].cat.add_categories(new_cat,inplace=True)
    pan_dataset_votes[col][mask] = new_cat

# turnout_var = "profile_turnout_2005"
past_vote_var = "profile_past_vote_2005"
new_vote_var = "2005GEVote"    

pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var].copy()
# BES_Panel["profile_past_vote_2005"].cat.categories
replace_dict = {'I would not vote':'Did not vote',
                'Conservative':'Conservative',
                'Labour':'Labour',
                'Liberal Democrat':'Liberal Democrat',
                'Scottish National Party (SNP)':'Scottish National Party',
                'Plaid Cymru':'Plaid Cymru',
                'United Kingdom Independence Party (UKIP)':'United Kingdom Independence Party (UKIP)',
                'Green Party':'Green Party',
                'British National Party (BNP)':'British National Party (BNP)',
                'Other': 'Other',
                10.0:'Ambiguous category',
                'Change UK- The Independent Group':'Veritas',
                'Brexit Party':'Respect',
                'An independent candidate':'Other',
                "Don't know":"Don't know",
               }
# 'An independent candidate' -> An independent candidate
# BES_Panel["profile_past_vote_2005"].replace(replace_dict)

pan_dataset_votes[new_vote_var] = pan_dataset_votes[new_vote_var].replace(replace_dict).astype('category')

too_young_to_vote = pan_dataset_ages["Age_at_W1"]<=(age_cut_off+8.818617385352498)
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote, "Too Young To Vote")

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]
# entried that don't actually seem real/to contain any data!
# nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")

# we have no information - probably because not covered in any wave
unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, "Unknown")    
    
    
# turnout_var = "profile_turnout_2010"
past_vote_var = "profile_past_vote_2010"
new_vote_var = "2010GEVote"

# # three different turnout variables!
# Didnt_vote_not_sure =BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
# Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["voted2015"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
# Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["genElecTurnoutRetroW6"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)

# lets assume that positive data (even "don't know"! in past_vote_var outranks recollection of voting/turnout)
# Didnt_vote_not_sure = Didnt_vote_not_sure & BES_Panel[past_vote_var].isnull()

pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var].copy()
# add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, "Did not vote")
# BES_Panel["profile_past_vote_2010"].cat.categories
replace_dict = {'I would not vote':'Did not vote',
                'Conservative':'Conservative',
                'Labour':'Labour',
                'Liberal Democrat':'Liberal Democrat',
                'Scottish National Party (SNP)':'Scottish National Party',
                'Plaid Cymru':'Plaid Cymru',
                'United Kingdom Independence Party (UKIP)':'United Kingdom Independence Party (UKIP)',
                'Green Party':'Green Party',
                'British National Party (BNP)':'British National Party (BNP)',
                'Other': 'Other',
                10.0:'Ambiguous category',
                'Change UK- The Independent Group':'Respect',
                'Brexit Party':'Other',
                "Don't know":"Don't know",
               }
# Brexit Party -> An independent candidate
pan_dataset_votes[new_vote_var] = pan_dataset_votes[new_vote_var].replace(replace_dict).astype('category')


too_young_to_vote = pan_dataset_ages["Age_at_W1"]<=(age_cut_off+3.8165639972621492)
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote, "Too Young To Vote")

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]
# entried that don't actually seem real/to contain any data!
# nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")

# we have no information - probably because not covered in any wave
unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, "Unknown")    
    
    
    
turnout_var = "profile_turnout_2015"
past_vote_var = "profile_past_vote_2015"
new_vote_var = "2015GEVote"

# three different turnout variables!
Didnt_vote_not_sure =BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["voted2015"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["genElecTurnoutRetroW6"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)

# lets assume that positive data (even "don't know"! in past_vote_var outranks recollection of voting/turnout)
Didnt_vote_not_sure = Didnt_vote_not_sure & BES_Panel[past_vote_var].isnull()

pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var]
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, "Did not vote")
# two non-voting variables for 2015!
# if "Did not vote" not in BES_Panel[past_vote_var].cat.categories:
#     BES_Panel[past_vote_var].cat.add_categories("Did not vote",inplace=True)
# # this is assumed to only be overwriting nans
# BES_Panel[past_vote_var][Didnt_vote_not_sure] = "Did not vote"

# BES_Panel[past_vote_var].value_counts()

too_young_to_vote = pan_dataset_ages["Age_at_W1"]<=(age_cut_off-1.1854893908281998)
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote, "Too Young To Vote")

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]
# entried that don't actually seem real/to contain any data!
# nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")

# we have no information - probably because not covered in any wave
unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, "Unknown")



turnout_var = "profile_turnout_2017"
past_vote_var = "profile_past_vote_2017"
new_vote_var = "2017GEVote"

# generalElectionVoteRetro2017W18     8063
# genElecTurnoutRetro2017W18          9369
# voted2017W19                       32177
# profile_past_vote_2017             70492
# profile_turnout_2017               70322


Didnt_vote_not_sure =BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["voted2017W19"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["genElecTurnoutRetro2017W18"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
# lets assume that positive data (even "don't know"! in past_vote_var outranks recollection of voting/turnout)
Didnt_vote_not_sure = Didnt_vote_not_sure & BES_Panel[past_vote_var].isnull()
# two non-voting variables for 2015!
pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var]
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, "Did not vote")

# if "Did not vote" not in BES_Panel[past_vote_var].cat.categories:
#     BES_Panel[past_vote_var].cat.add_categories("Did not vote",inplace=True)
# # this is assumed to only be overwriting nans
# BES_Panel[past_vote_var][Didnt_vote_not_sure] = "Did not vote"

too_young_to_vote = pan_dataset_ages["Age_at_W12"]<=age_cut_off
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote, "Too Young To Vote")

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]
# entried that don't actually seem real/to contain any data!
# nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")

# we have no information - probably because not covered in any wave
unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, "Unknown")


# BES_Panel[past_vote_var].value_counts()

turnout_var = "genElecTurnoutRetroW19"
past_vote_var = "pastvote_ge_2019W19"
new_vote_var = "2019GEVote"

Didnt_vote_not_sure =BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)

# lets assume that positive data (even "don't know"! in past_vote_var outranks recollection of voting/turnout)
Didnt_vote_not_sure = Didnt_vote_not_sure & BES_Panel[past_vote_var].isnull()
# two non-voting variables for 2015!
pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var]
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, "Did not vote")

too_young_to_vote = pan_dataset_ages["Age_at_W18"]<=age_cut_off
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote, "Too Young To Vote")

# if "Did not vote" not in pan_dataset_votes[new_vote_var].cat.categories:
#     pan_dataset_votes[new_vote_var].cat.add_categories("Did not vote",inplace=True)
# # this is assumed to only be overwriting nans
# pan_dataset_votes[new_vote_var][Didnt_vote_not_sure] = "Did not vote"

pan_dataset_votes[new_vote_var] = pan_dataset_votes[new_vote_var].cat.rename_categories({"Green":"Green Party"})
pan_dataset_votes[new_vote_var] = pan_dataset_votes[new_vote_var].cat.add_categories({'United Kingdom Independence Party (UKIP)'})

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]
# entried that don't actually seem real/to contain any data!
# nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")
# if "Not Present" not in pan_dataset_votes[new_vote_var+"_complete"].cat.categories:
#     pan_dataset_votes[new_vote_var+"_complete"].cat.add_categories("Not Present",inplace=True)
# pan_dataset_votes[new_vote_var+"_complete"][nowtsW19] = "Not Present"



replace_dict = {'An independent candidate':'Other'}
# there's 3 - and only - UKIP VI people in this ~700 ppl - and no UKIP category
# seems a bit dumb to add a category for just 3 ppl ...
# unless you're going to create a coherent set of categories for all votes???
last_chunk_unassigned = (pan_dataset_votes[new_vote_var+"_complete"].isnull() & BES_Panel["wt_new_W19_result"].notnull())
pan_dataset_votes[new_vote_var+"_complete"][last_chunk_unassigned] = \
    BES_Panel["generalElectionVoteW19"][last_chunk_unassigned].astype('object').replace(replace_dict)\
        .astype('category').cat.set_categories(pan_dataset_votes[new_vote_var+"_complete"].cat.categories,ordered =True).values

# we have no information - probably because not covered in any wave
unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, "Unknown")

# too_young_to_vote = pan_dataset_ages["Age_at_W18"]<=18
# add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", too_young_to_vote, "Too Young To Vote")


# BES_Panel[past_vote_var].value_counts()

turnout_var = "profile_eurefturnout"
past_vote_var = "profile_eurefvote"
new_vote_var = "2016EUVote"

Didnt_vote_not_sure = BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)

# lets assume that positive data (even "don't know"! in past_vote_var outranks recollection of voting/turnout)
Didnt_vote_not_sure = Didnt_vote_not_sure & BES_Panel[past_vote_var].isnull()

pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var]
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, "Did not vote")

# # two non-voting variables for 2015!
# if "Did not vote" not in BES_Panel[past_vote_var].cat.categories:
#     BES_Panel[past_vote_var].cat.add_categories("Did not vote",inplace=True)
# # this is assumed to only be overwriting nans
# BES_Panel[past_vote_var][Didnt_vote_not_sure] = "Did not vote"

mask = BES_Panel["pastvote_EURefW17"].notnull() & pan_dataset_votes[new_vote_var].isnull()

# BES_Panel["pastvote_EURefW17"][mask].replace(replace_dict).astype('category').cat.set_categories(BES_Panel["profile_eurefvote"].cat.categories)
replace_dict = {"Male":'Stay/remain in the EU',"Female":'Leave the EU',3.0:'Did not vote',4.0:"Don't know"}
pan_dataset_votes[new_vote_var][mask] = \
    BES_Panel["pastvote_EURefW17"][mask].astype('object').replace(replace_dict)\
        .astype('category').cat.set_categories(['Stay/remain in the EU', 'Leave the EU', "Don't know", 'Did not vote'],
                                               ordered=True).values
# BES_Panel["profile_eurefvote"].cat.categories

too_young_to_vote = pan_dataset_ages["Age_at_W8"]<=age_cut_off
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote, "Too Young To Vote")

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]
# entried that don't actually seem real/to contain any data!

add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")

# we have no information - probably because not covered in any wave
unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, "Unknown")


turnout_var = "euroTurnoutRetroW2"
past_vote_var = "euroElectionVoteW2"
new_vote_var = "2014EEVote"

# euroTurnoutW1          30542
# euroTurnoutRetroW2     30262
# euroElectionVoteW1             30537
# euroElectionVoteW2             22188

Didnt_vote_not_sure =BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["euroTurnoutW1"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
# lets assume that positive data (even "don't know"! in past_vote_var outranks recollection of voting/turnout)
Didnt_vote_not_sure = Didnt_vote_not_sure & BES_Panel[past_vote_var].isnull()
# two non-voting variables for 2015!
pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var]
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, "Did not vote")

too_young_to_vote = pan_dataset_ages["Age_at_W1"]<=age_cut_off
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote, "Too Young To Vote")

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]

add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")

last_chunk_unassigned = ( pan_dataset_votes[new_vote_var+"_complete"].isnull() & BES_Panel["wave2"] )
pan_dataset_votes[new_vote_var+"_complete"][last_chunk_unassigned] = \
    BES_Panel["euroElectionVoteW2"][last_chunk_unassigned].astype('object').replace(replace_dict)\
        .astype('category').cat.set_categories(pan_dataset_votes[new_vote_var+"_complete"].cat.categories,ordered =True).values

# we have no information - probably because not covered in any wave
unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, "Unknown")


turnout_var = "euroTurnoutRetroW16"
past_vote_var = "euroElectionVoteW16"
new_vote_var = "2019EEVote"

# euroTurnoutRetroW16    37959
# euroElectionVoteW16            25837
# euroElectionVoteNonVoterW16    12122

Didnt_vote_not_sure =BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
# Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["euroTurnoutW1"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
# lets assume that positive data (even "don't know"! in past_vote_var outranks recollection of voting/turnout)
Didnt_vote_not_sure = Didnt_vote_not_sure & BES_Panel[past_vote_var].isnull()
# two non-voting variables for 2015!
pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var]
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, "Did not vote")

too_young_to_vote = pan_dataset_ages["Age_at_W16"]<=age_cut_off
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote, "Too Young To Vote")

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]

add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")

# we have no information - probably because not covered in any wave
unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, "Unknown")



In [488]:
pan_dataset_votes["2014EEVote"+"_complete"].value_counts(dropna=False)

Unknown                                     60988
Did not vote                                 8197
United Kingdom Independence Party (UKIP)     6365
Too Young To Vote                            5258
Labour                                       5074
Conservative                                 3899
Green Party                                  2521
Scottish National Party (SNP)                1443
Liberal Democrat                             1379
Not Present                                   629
Other                                         565
Plaid Cymru                                   437
Don't know                                    308
British National Party (BNP)                   95
Name: 2014EEVote_complete, dtype: int64

In [489]:
pan_dataset_votes["2019EEVote"+"_complete"].value_counts(dropna=False)

Unknown                                     58536
Did not vote                                12034
Brexit Party                                 9229
Liberal Democrat                             5369
Green Party                                  3517
Labour                                       2510
Conservative                                 1715
Scottish National Party (SNP)                 945
Change UK- The Independent Group              882
Not Present                                   629
Other                                         526
United Kingdom Independence Party (UKIP)      399
Don't know                                    379
Plaid Cymru                                   294
Too Young To Vote                             194
Name: 2019EEVote_complete, dtype: int64

In [469]:
pan_dataset_votes["2005GEVote_complete"].value_counts(dropna=False)

Unknown                                     25473
Too Young To Vote                           19789
Labour                                      15323
Conservative                                11666
Liberal Democrat                             9423
Did not vote                                 4783
Scottish National Party                      3792
Other                                        1770
Don't know                                   1692
United Kingdom Independence Party (UKIP)      684
Not Present                                   629
Plaid Cymru                                   625
Green Party                                   614
British National Party (BNP)                  505
Ambiguous category                            283
Respect                                       104
Veritas                                         3
Name: 2005GEVote_complete, dtype: int64

In [470]:
pan_dataset_votes["2010GEVote_complete"].value_counts(dropna=False)

Unknown                                     23666
Labour                                      16448
Conservative                                14808
Liberal Democrat                            12653
Too Young To Vote                           11716
Scottish National Party                      6067
Did not vote                                 4436
Other                                        1376
United Kingdom Independence Party (UKIP)     1211
Don't know                                    925
British National Party (BNP)                  841
Plaid Cymru                                   800
Ambiguous category                            728
Green Party                                   663
Not Present                                   629
Respect                                       191
Name: 2010GEVote_complete, dtype: int64

In [471]:
pan_dataset_votes["2015GEVote_complete"].value_counts(dropna=False)

Conservative                                23223
Labour                                      21820
Unknown                                     12616
Did not vote                                10971
United Kingdom Independence Party (UKIP)     7424
Liberal Democrat                             5950
Green Party                                  3817
Scottish National Party (SNP)                3813
Too Young To Vote                            3005
Don't know                                   2206
Other                                        1084
Not Present                                   629
Plaid Cymru                                   535
British National Party (BNP)                   65
Name: 2015GEVote_complete, dtype: int64

In [472]:
pan_dataset_votes["2017GEVote_complete"].value_counts(dropna=False)

Unknown                                     26009
Conservative                                23716
Labour                                      23624
Did not vote                                 8975
Liberal Democrat                             5245
Scottish National Party (SNP)                2515
United Kingdom Independence Party (UKIP)     1482
Too Young To Vote                            1403
Green Party                                  1274
Don't know                                   1173
Other                                         716
Not Present                                   629
Plaid Cymru                                   396
British National Party (BNP)                    1
Name: 2017GEVote_complete, dtype: int64

In [473]:
pan_dataset_votes["2019GEVote"].value_counts(dropna=False)

NaN                                         65715
Conservative                                12666
Labour                                       8244
Liberal Democrat                             3579
Did not vote                                 3159
Scottish National Party (SNP)                1249
Green Party                                   858
Brexit Party                                  722
Other                                         553
Don't know                                    231
Plaid Cymru                                   182
United Kingdom Independence Party (UKIP)        0
Too Young To Vote                               0
Name: 2019GEVote, dtype: int64

In [474]:
pan_dataset_votes["2019GEVote_complete"].value_counts(dropna=False)

Unknown                                     64352
Conservative                                12890
Labour                                       8453
Liberal Democrat                             3648
Did not vote                                 3159
Scottish National Party (SNP)                1281
Green Party                                   894
Brexit Party                                  746
Not Present                                   629
Other                                         603
Don't know                                    317
Plaid Cymru                                   183
United Kingdom Independence Party (UKIP)        3
Too Young To Vote                               0
Name: 2019GEVote_complete, dtype: int64

In [490]:
pan_dataset_votes.stack().unique()

array(['Plaid Cymru', 'British National Party (BNP)', 'Labour',
       'Brexit Party', 'Leave the EU',
       'United Kingdom Independence Party (UKIP)', 'Conservative',
       'Stay/remain in the EU', 'Did not vote', 'Unknown', "Don't know",
       'Too Young To Vote', 'Liberal Democrat', 'Scottish National Party',
       'Other', 'Green Party', 'Ambiguous category',
       'Scottish National Party (SNP)', 'Respect',
       'Change UK- The Independent Group', 'Not Present', 'Veritas'],
      dtype=object)

In [491]:
pan_dataset_votes.to_pickle(BES_small_data_files + "pan_dataset_votes"+".zip", compression='zip')

In [383]:
search(BES_Panel,"euroTurnout")

euroTurnoutW1          30542
euroTurnoutRetroW2     30262
euroTurnoutRetroW16    37959
dtype: int64

In [381]:
search(BES_Panel,"euroElection")

euroElectionVoteW1             30537
euroElectionVoteW2             22188
euroElectionVoteW16            25837
euroElectionVoteNonVoterW16    12122
dtype: int64

In [379]:
search(BES_Panel,"pastvote")

euRefpastVoteW11        27999
euRefpastVoteW12        31173
euRefpastVoteW13        28288
euRefpastVoteW14         3822
euRefpastVoteW15        27007
euRefpastVoteW17         2643
pastvote_EURefW17       34352
pastvote_eup_2019W18    27133
pastvote_ge_2019W19     31443
pastvote_eup_2019W19    23186
dtype: int64

In [306]:
search(BES_Panel,"2010")

profile_past_vote_2010    65581
dtype: int64

In [340]:
corrs = pd.get_dummies(BES_Panel[["profile_past_vote_2005","profile_past_vote_2010","profile_past_vote_2015"]]).corr()
corrs[[x for x in corrs.columns if "profile_past_vote_2010" in x]].idxmax(axis=1)

profile_past_vote_2005_I would not vote                                      profile_past_vote_2010_I would not vote
profile_past_vote_2005_Conservative                                              profile_past_vote_2010_Conservative
profile_past_vote_2005_Labour                                                          profile_past_vote_2010_Labour
profile_past_vote_2005_Liberal Democrat                                      profile_past_vote_2010_Liberal Democrat
profile_past_vote_2005_Scottish National Party (SNP)               profile_past_vote_2010_Scottish National Party...
profile_past_vote_2005_Plaid Cymru                                                profile_past_vote_2010_Plaid Cymru
profile_past_vote_2005_United Kingdom Independence Party (UKIP)    profile_past_vote_2010_United Kingdom Independ...
profile_past_vote_2005_Green Party                                                profile_past_vote_2010_Green Party
profile_past_vote_2005_British National Party (BNP)             

In [359]:
list(search(BES_reduced_with_na,"ptv.*W1_").index)

['ptvConW1__Very likely',
 'ptvLabW1__Very likely',
 'ptvLDW1__Very likely',
 'ptvSNPW1__Very likely',
 'ptvPCW1__Very likely',
 'ptvUKIPW1__Very likely',
 'ptvGrnW1__Very likely',
 'ptvBNPW1__Very likely']

In [361]:
corrs['ptvConW1__Very likely', 'ptvLabW1__Very likely',

ptvConW1__Very likely                                              1.000000
ptvLabW1__Very likely                                             -0.439090
ptvLDW1__Very likely                                               0.132162
ptvSNPW1__Very likely                                             -0.360742
ptvPCW1__Very likely                                              -0.167955
ptvUKIPW1__Very likely                                             0.235519
ptvGrnW1__Very likely                                             -0.215325
ptvBNPW1__Very likely                                              0.053291
profile_past_vote_2010_I would not vote                           -0.099291
profile_past_vote_2010_Conservative                                0.650714
profile_past_vote_2010_Labour                                     -0.351950
profile_past_vote_2010_Liberal Democrat                           -0.157116
profile_past_vote_2010_Scottish National Party (SNP)              -0.135695
profile_past

In [362]:
# pd.get_dummies(BES_Panel["profile_past_vote_2010"])
corrs= pd.concat([BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W1_").index].astype('float32'),
           pd.get_dummies(BES_Panel[["profile_past_vote_2010"]])],axis=1).corr()
# corrs
corrs[list(search(BES_reduced_with_na,"ptv.*W1_").index)].idxmax(axis=1)

ptvConW1__Very likely                                               ptvConW1__Very likely
ptvLabW1__Very likely                                               ptvLabW1__Very likely
ptvLDW1__Very likely                                                 ptvLDW1__Very likely
ptvSNPW1__Very likely                                               ptvSNPW1__Very likely
ptvPCW1__Very likely                                                 ptvPCW1__Very likely
ptvUKIPW1__Very likely                                             ptvUKIPW1__Very likely
ptvGrnW1__Very likely                                               ptvGrnW1__Very likely
ptvBNPW1__Very likely                                               ptvBNPW1__Very likely
profile_past_vote_2010_I would not vote                             ptvBNPW1__Very likely
profile_past_vote_2010_Conservative                                 ptvConW1__Very likely
profile_past_vote_2010_Labour                                       ptvLabW1__Very likely
profile_pa

In [363]:
# pd.get_dummies(BES_Panel["profile_past_vote_2010"])
corrs= pd.concat([BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W1_").index].astype('float32'),
           pd.get_dummies(BES_Panel[["profile_past_vote_2005"]])],axis=1).corr()
# corrs
corrs[list(search(BES_reduced_with_na,"ptv.*W1_").index)].idxmax(axis=1)

ptvConW1__Very likely                                               ptvConW1__Very likely
ptvLabW1__Very likely                                               ptvLabW1__Very likely
ptvLDW1__Very likely                                                 ptvLDW1__Very likely
ptvSNPW1__Very likely                                               ptvSNPW1__Very likely
ptvPCW1__Very likely                                                 ptvPCW1__Very likely
ptvUKIPW1__Very likely                                             ptvUKIPW1__Very likely
ptvGrnW1__Very likely                                               ptvGrnW1__Very likely
ptvBNPW1__Very likely                                               ptvBNPW1__Very likely
profile_past_vote_2005_I would not vote                             ptvBNPW1__Very likely
profile_past_vote_2005_Conservative                                 ptvConW1__Very likely
profile_past_vote_2005_Labour                                       ptvLabW1__Very likely
profile_pa

In [364]:
# pd.get_dummies(BES_Panel["profile_past_vote_2010"])
corrs= pd.concat([BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W5_").index].astype('float32'),
           pd.get_dummies(BES_Panel[["profile_past_vote_2015"]])],axis=1).corr()
# corrs
corrs[list(search(BES_reduced_with_na,"ptv.*W5_").index)].idxmax(axis=1)

ptvConW5__Very likely                                               ptvConW5__Very likely
ptvLabW5__Very likely                                               ptvLabW5__Very likely
ptvLDW5__Very likely                                                 ptvLDW5__Very likely
ptvSNPW5__Very likely                                               ptvSNPW5__Very likely
ptvPCW5__Very likely                                                 ptvPCW5__Very likely
ptvUKIPW5__Very likely                                             ptvUKIPW5__Very likely
ptvGrnW5__Very likely                                               ptvGrnW5__Very likely
ptvBNPW5__Very likely                                               ptvBNPW5__Very likely
profile_past_vote_2015_Conservative                                 ptvConW5__Very likely
profile_past_vote_2015_Labour                                       ptvLabW5__Very likely
profile_past_vote_2015_Liberal Democrat                              ptvLDW5__Very likely
profile_pa

In [ ]:
corrs = pd.get_dummies(BES_Panel[["profile_past_vote_2005","profile_past_vote_2010","profile_past_vote_2015",]]).corr()
corrs[[x for x in corrs.columns if "profile_past_vote_2010" in x]].idxmax(axis=1)

In [365]:
BES_Panel["profile_past_vote_2010"].value_counts()

Conservative                                17934
Labour                                      16643
Liberal Democrat                            12853
Scottish National Party (SNP)                6158
I would not vote                             5001
United Kingdom Independence Party (UKIP)     1220
Brexit Party                                 1126
Don't know                                    952
British National Party (BNP)                  868
Plaid Cymru                                   829
10.0                                          741
Green Party                                   686
Other                                         371
Change UK- The Independent Group              199
Name: profile_past_vote_2010, dtype: int64

In [366]:
BES_Panel["profile_past_vote_2005"].value_counts()
Change UK- The Independent Group                3 -> veritas?
Brexit Party                                  116 -> respect

Conservative                                16407
Labour                                      15435
Liberal Democrat                             9582
I would not vote                             4791
Scottish National Party (SNP)                3857
Don't know                                   1694
An independent candidate                     1611
United Kingdom Independence Party (UKIP)      691
Plaid Cymru                                   647
Green Party                                   622
British National Party (BNP)                  523
Other                                         357
10.0                                          295
Brexit Party                                  116
Change UK- The Independent Group                3
Name: profile_past_vote_2005, dtype: int64

In [368]:
BES_Panel["profile_past_vote_2010"].cat.categories

Index([                        'I would not vote',
                                   'Conservative',
                                         'Labour',
                               'Liberal Democrat',
                  'Scottish National Party (SNP)',
                                    'Plaid Cymru',
       'United Kingdom Independence Party (UKIP)',
                                    'Green Party',
                   'British National Party (BNP)',
                                          'Other',
                                             10.0,
               'Change UK- The Independent Group',
                                   'Brexit Party',
                                     'Don't know'],
      dtype='object')

In [367]:
BES_Panel["profile_past_vote_2005"].cat.categories

Index([                        'I would not vote',
                                   'Conservative',
                                         'Labour',
                               'Liberal Democrat',
                  'Scottish National Party (SNP)',
                                    'Plaid Cymru',
       'United Kingdom Independence Party (UKIP)',
                                    'Green Party',
                   'British National Party (BNP)',
                                          'Other',
                                             10.0,
               'Change UK- The Independent Group',
                                   'Brexit Party',
                       'An independent candidate',
                                     'Don't know'],
      dtype='object')

In [370]:
BES_Panel["profile_past_vote_2005"].cat.categories
replace_dict = {'I would not vote':'Did not vote',
                'Conservative':'Conservative Party',
                'Labour':'Labour Party',
                'Liberal Democrat':'Liberal Democrats',
                'Scottish National Party (SNP)':'Scottish National Party',
                'Plaid Cymru':'Plaid Cymru',
                'United Kingdom Independence Party (UKIP)':'United Kingdom Independence Party (UKIP)',
                'Green Party':'Green Party',
                'British National Party (BNP)':'British National Party (BNP)',
                'Other': 'Some other party',
                10.0:'Ambiguous category',
                'Change UK- The Independent Group':'Veritas',
                'Brexit Party':'Respect',
                'An independent candidate':'An independent candidate',
                "Don't know":"Don't know",
               }
BES_Panel["profile_past_vote_2005"].replace(replace_dict)

Conservative Party                          16407
Labour Party                                15435
Liberal Democrats                            9582
Did not vote                                 4791
Scottish National Party                      3857
Don't know                                   1694
An independent candidate                     1611
United Kingdom Independence Party (UKIP)      691
Plaid Cymru                                   647
Green Party                                   622
British National Party (BNP)                  523
Some other party                              357
Ambiguous category                            295
Respect                                       116
Veritas                                         3
Name: profile_past_vote_2005, dtype: int64

In [310]:
BES_Panel["profile_past_vote_2010"].cat.categories
replace_dict = {'I would not vote':'Did not vote',
                'Conservative':'Conservative Party',
                'Labour':'Labour Party',
                'Liberal Democrat':'Liberal Democrats',
                'Scottish National Party (SNP)':'Scottish National Party',
                'Plaid Cymru':'Plaid Cymru',
                'United Kingdom Independence Party (UKIP)':'United Kingdom Independence Party (UKIP)',
                'Green Party':'Green Party',
                'British National Party (BNP)':'British National Party (BNP)',
                'Other': 'Some other party',
                10.0:'Ambiguous category',
                'Change UK- The Independent Group':'Respect',
                'Brexit Party':'An independent candidate',
                "Don't know":"Don't know",
               }
BES_Panel["profile_past_vote_2010"].replace(replace_dict)

Index([                        'I would not vote',
                                   'Conservative',
                                         'Labour',
                               'Liberal Democrat',
                  'Scottish National Party (SNP)',
                                    'Plaid Cymru',
       'United Kingdom Independence Party (UKIP)',
                                    'Green Party',
                   'British National Party (BNP)',
                                          'Other',
                                             10.0,
               'Change UK- The Independent Group',
                                   'Brexit Party',
                                     'Don't know'],
      dtype='object')

In [315]:
search(BES_reduced_with_na,"profile_past_vote_2010")

profile_past_vote_2010_Conservative                                59257
profile_past_vote_2010_Labour                                      59257
profile_past_vote_2010_Liberal Democrat                            59257
profile_past_vote_2010_Scottish National Party (SNP)               59257
profile_past_vote_2010_United Kingdom Independence Party (UKIP)    59257
profile_past_vote_2010_Brexit Party                                59257
profile_past_vote_2010_British National Party (BNP)                59257
profile_past_vote_2010_Plaid Cymru                                 59257
profile_past_vote_2010_10.0                                        59257
profile_past_vote_2010_Green Party                                 59257
profile_past_vote_2010_Change UK- The Independent Group            59257
dtype: int64

In [322]:
BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W1__|profile_past_vote_2010").index].astype('float32').corr()[search(BES_reduced_with_na,"ptv.*W1__").index]

,ptvConW1__Very likely,ptvLabW1__Very likely,ptvLDW1__Very likely,ptvSNPW1__Very likely,ptvPCW1__Very likely,ptvUKIPW1__Very likely,ptvGrnW1__Very likely,ptvBNPW1__Very likely
ptvConW1__Very likely,1.000000,-0.439090,0.132162,-0.360742,-0.167955,0.235519,-0.215325,0.053291
ptvLabW1__Very likely,-0.439090,1.000000,0.167575,-0.194455,0.156821,-0.294187,0.247891,-0.066843
ptvLDW1__Very likely,0.132162,0.167575,1.000000,-0.094208,0.226703,-0.132099,0.306397,-0.002038
ptvSNPW1__Very likely,-0.360742,-0.194455,-0.094208,1.000000,NaN,-0.106209,0.288918,0.015724
ptvPCW1__Very likely,-0.167955,0.156821,0.226703,NaN,1.000000,-0.103549,0.339524,0.007574
ptvUKIPW1__Very likely,0.235519,-0.294187,-0.132099,-0.106209,-0.103549,1.000000,-0.068596,0.427586
ptvGrnW1__Very likely,-0.215325,0.247891,0.306397,0.288918,0.339524,-0.068596,1.000000,0.100275
ptvBNPW1__Very likely,0.053291,-0.066843,-0.002038,0.015724,0.007574,0.427586,0.100275,1.000000
profile_past_vote_2010_Conservative,0.691073,-0.496065,-0.119539,-0.307137,-0.276412,0.263719,-0.280498,0.050916
profile_past_vote_2010_Labour,-0.408518,0.601367,-0.125881,-0.206074,-0.077407,-0.224659,0.034990,-0.059540


In [326]:
BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W13__|profile_past_vote_2010").index].astype('float32').corr()[search(BES_reduced_with_na,"ptv.*W13__").index]

,ptvConW13__Very likely,ptvLabW13__Very likely,ptvLDW13__Very likely,ptvSNPW13__Very likely,ptvPCW13__Very likely,ptvUKIPW13__Very likely,ptvGrnW13__Very likely
ptvConW13__Very likely,1.000000,-0.644215,-0.206287,-0.593521,-0.365975,0.333739,-0.423337
ptvLabW13__Very likely,-0.644215,1.000000,0.328559,0.014575,0.301948,-0.333641,0.423734
ptvLDW13__Very likely,-0.206287,0.328559,1.000000,-0.077990,0.358562,-0.268544,0.420888
ptvSNPW13__Very likely,-0.593521,0.014575,-0.077990,1.000000,NaN,-0.229320,0.525867
ptvPCW13__Very likely,-0.365975,0.301948,0.358562,NaN,1.000000,-0.210304,0.422959
ptvUKIPW13__Very likely,0.333739,-0.333641,-0.268544,-0.229320,-0.210304,1.000000,-0.187684
ptvGrnW13__Very likely,-0.423337,0.423734,0.420888,0.525867,0.422959,-0.187684,1.000000
profile_past_vote_2010_Conservative,0.290167,-0.249445,-0.112997,-0.210351,-0.267745,0.102860,-0.163129
profile_past_vote_2010_Labour,0.074732,0.023380,-0.121327,-0.171865,-0.057046,-0.020494,-0.096643
profile_past_vote_2010_Liberal Democrat,-0.249964,0.260280,0.185332,-0.054602,0.088945,-0.127700,0.147939


In [330]:
BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W1__").index].astype('float32').corrwith((BES_Panel["profile_past_vote_2010"]=="Other"))

ptvConW1__Very likely    -0.027092
ptvLabW1__Very likely    -0.031455
ptvLDW1__Very likely     -0.026559
ptvSNPW1__Very likely    -0.004223
ptvPCW1__Very likely     -0.001329
ptvUKIPW1__Very likely    0.010653
ptvGrnW1__Very likely     0.004426
ptvBNPW1__Very likely     0.005244
dtype: float64

In [ ]:
BES_Panel[ BES_Panel["profile_past_vote_2010"]=="Change UK- The Independent Group" ][search(BES_Panel,"wave\d+").index].sum()

In [325]:
BES_Panel[ BES_Panel["profile_past_vote_2010"]=="Brexit Party" ][search(BES_Panel,"wave\d+").index].sum()

wave1      10
wave2       9
wave3      14
wave4      16
wave5      17
wave6      13
wave7     225
wave8     388
wave9     325
wave10    500
wave11    582
wave12    730
wave13    638
wave14    617
wave15    494
wave16    463
wave17    406
wave18    419
wave19    354
dtype: int64

In [331]:
BES_Panel["profile_past_vote_2010"].value_counts().sort_index()

I would not vote                             5001
Conservative                                17934
Labour                                      16643
Liberal Democrat                            12853
Scottish National Party (SNP)                6158
Plaid Cymru                                   829
United Kingdom Independence Party (UKIP)     1220
Green Party                                   686
British National Party (BNP)                  868
Other                                         371
10.0                                          741
Change UK- The Independent Group              199
Brexit Party                                 1126
Don't know                                    952
Name: profile_past_vote_2010, dtype: int64

In [332]:
BES_Panel["profile_past_vote_2015"].value_counts().sort_index()

Conservative                                23284
Labour                                      21943
Liberal Democrat                             5977
Scottish National Party (SNP)                3825
Plaid Cymru                                   535
United Kingdom Independence Party (UKIP)     7434
Green Party                                  3835
British National Party (BNP)                   66
Other                                        1087
Don't know                                   2591
Did not vote                                12491
Name: profile_past_vote_2015, dtype: int64

In [313]:
BES_Panel["profile_past_vote_2015"].value_counts()

Conservative                                23284
Labour                                      21943
Did not vote                                12491
United Kingdom Independence Party (UKIP)     7434
Liberal Democrat                             5977
Green Party                                  3835
Scottish National Party (SNP)                3825
Don't know                                   2591
Other                                        1087
Plaid Cymru                                   535
British National Party (BNP)                   66
Name: profile_past_vote_2015, dtype: int64

In [314]:
BES_Panel["profile_past_vote_2005"].value_counts()

Conservative                                16407
Labour                                      15435
Liberal Democrat                             9582
I would not vote                             4791
Scottish National Party (SNP)                3857
Don't know                                   1694
An independent candidate                     1611
United Kingdom Independence Party (UKIP)      691
Plaid Cymru                                   647
Green Party                                   622
British National Party (BNP)                  523
Other                                         357
10.0                                          295
Brexit Party                                  116
Change UK- The Independent Group                3
Name: profile_past_vote_2005, dtype: int64

In [268]:
pd.crosstab(BES_Panel["euRefTurnoutW8"], BES_Panel["profile_eurefturnout"])

profile_eurefturnout,"No, did not vote","Yes, voted",Don't know
euRefTurnoutW8,,,
Very unlikely that I would vote,599,151,12
Fairly unlikely,176,149,8
Neither likely nor unlikely,220,326,22
Fairly likely,251,1594,20
Very likely that I would vote,369,23143,43
Don't know,168,257,56


In [255]:
search(BES_Panel,"euref",mask = BES_Panel["profile_eurefvote"].isnull()).sort_values().tail(50)

euRefpastVoteW14                     0
euRefDoOverW15                       0
euRefVoteW15                         0
goodConductEURefW15                  0
euRefTurnoutRetroW15                 0
euRefpastVoteW15                     0
euRefTurnoutRetroW17                 0
euRefpastVoteW17                     0
euRefpastVoteW13                     0
euRefDoOverW10                       0
profile_eurefturnout                 0
euRefFinalPostW9                     0
euRefVoteW9                          0
euRefVoteHypotheticalW9              0
euRefTurnoutRetroW9                  0
goodConductEURefW9                   0
euRefVoteW16                         5
euRefDoOverW16                       5
EUrefsatisfactionControl_1W19       28
euRefVotePostW8                    207
euRefVoteSqueezeW8                 225
euRefVoteW19                       271
euRefDoOverW19                     271
pastvote_EURefW17                  282
euRefVoteW17                       283
euRefDoOverW17           

In [256]:
BES_Panel["pastvote_EURefW17"][BES_Panel["profile_eurefvote"].isnull()].value_counts()

Male      135
Female    124
3.0        19
4.0         4
Name: pastvote_EURefW17, dtype: int64

In [258]:
BES_Panel["profile_eurefvote"].value_counts()

Stay/remain in the EU    36286
Leave the EU             31721
Did not vote             10997
Don't know                 507
Name: profile_eurefvote, dtype: int64

In [263]:
BES_Panel["pastvote_EURefW17"].value_counts()
replace_dict = {"Male":'Stay/remain in the EU',"Female":'Leave the EU',3.0:'Did not vote',4.0:"Don't know"}
BES_Panel["pastvote_EURefW17"].replace(replace_dict).astype('category').cat.set_categories(BES_Panel["profile_eurefvote"].cat.categories)

Stay/remain in the EU    15266
Leave the EU             14843
Did not vote              3714
Don't know                 529
Name: pastvote_EURefW17, dtype: int64

In [260]:
BES_Panel["pastvote_EURefW17"][BES_Panel["profile_eurefvote"] == "Stay/remain in the EU"].value_counts()

Male      14469
Female      164
3.0          97
4.0          57
Name: pastvote_EURefW17, dtype: int64

In [254]:
search(BES_Panel,"euref",mask = BES_Panel["profile_eurefturnout"].isnull()).sort_values().tail(50)

euRefpastVoteW14                     0
euRefDoOverW15                       0
euRefVoteW15                         0
goodConductEURefW15                  0
euRefTurnoutRetroW15                 0
euRefpastVoteW15                     0
euRefTurnoutRetroW17                 0
euRefpastVoteW17                     0
euRefpastVoteW13                     0
euRefDoOverW10                       0
profile_eurefturnout                 0
euRefFinalPostW9                     0
euRefVoteW9                          0
euRefVoteHypotheticalW9              0
euRefTurnoutRetroW9                  0
goodConductEURefW9                   0
euRefVoteW16                         5
euRefDoOverW16                       5
EUrefsatisfactionControl_1W19       28
euRefVotePostW8                    207
euRefVoteSqueezeW8                 225
euRefVoteW19                       271
euRefDoOverW19                     271
pastvote_EURefW17                  282
euRefVoteW17                       283
euRefDoOverW17           

In [ ]:
pastvote_EURefW17

In [248]:
pan_dataset_votes["2017GEVote"+"_complete"].value_counts(dropna=False)

Unknown                                     26037
Labour                                      23817
Conservative                                23760
Did not vote                                10041
Liberal Democrat                             5275
Scottish National Party (SNP)                2531
United Kingdom Independence Party (UKIP)     1484
Green Party                                  1285
Don't know                                   1179
Other                                         723
Not Present                                   629
Plaid Cymru                                   396
British National Party (BNP)                    1
Name: 2017GEVote_complete, dtype: int64

In [243]:
turnout_var = "profile_turnout_2017"
past_vote_var = "profile_past_vote_2017"
new_vote_var = "2017GEVote"

# generalElectionVoteRetro2017W18     8063
# genElecTurnoutRetro2017W18          9369
# voted2017W19                       32177
# profile_past_vote_2017             70492
# profile_turnout_2017               70322


Didnt_vote_not_sure =BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["voted2017W19"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel["genElecTurnoutRetro2017W18"].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)
# lets assume that positive data (even "don't know"! in past_vote_var outranks recollection of voting/turnout)
Didnt_vote_not_sure = Didnt_vote_not_sure & BES_Panel[past_vote_var].isnull()
# two non-voting variables for 2015!
pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var]
add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, "Did not vote")

# if "Did not vote" not in BES_Panel[past_vote_var].cat.categories:
#     BES_Panel[past_vote_var].cat.add_categories("Did not vote",inplace=True)
# # this is assumed to only be overwriting nans
# BES_Panel[past_vote_var][Didnt_vote_not_sure] = "Did not vote"

pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]
# entries that don't actually seem real/to contain any data!
nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])
add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, "Not Present")

too_young_to_vote = pan_dataset_ages["Age_at_W12"]<=17

In [244]:
pan_dataset_votes[new_vote_var+"_complete"][too_young_to_vote].value_counts()

Did not vote                                666
Labour                                       29
Conservative                                  7
Liberal Democrat                              4
Green Party                                   2
Scottish National Party (SNP)                 2
Don't know                                    1
Other                                         1
Not Present                                   0
British National Party (BNP)                  0
United Kingdom Independence Party (UKIP)      0
Plaid Cymru                                   0
Name: 2017GEVote_complete, dtype: int64

In [210]:
BES_Panel["profile_past_vote_2017"][BES_Panel["generalElectionVoteRetro2017W18"].notnull()].value_counts()


Conservative                                3422
Labour                                      2803
Liberal Democrat                             650
Scottish National Party (SNP)                316
Did not vote                                 289
United Kingdom Independence Party (UKIP)     199
Green Party                                  154
Don't know                                    98
Other                                         87
Plaid Cymru                                   45
British National Party (BNP)                   0
Name: profile_past_vote_2017, dtype: int64

In [211]:
BES_Panel["generalElectionVoteRetro2017W18"].value_counts()

Conservative                                3478
Labour                                      2409
Liberal Democrat                             793
United Kingdom Independence Party (UKIP)     416
Scottish National Party (SNP)                336
Don't know                                   236
Green Party                                  224
Other                                        119
Plaid Cymru                                   52
Name: generalElectionVoteRetro2017W18, dtype: int64

In [212]:
BES_Panel["generalElectionVoteRetro2017W18"][BES_Panel["profile_past_vote_2017"]=="Did not vote"].value_counts()


Conservative                                106
Labour                                       73
Liberal Democrat                             38
United Kingdom Independence Party (UKIP)     34
Scottish National Party (SNP)                17
Don't know                                   11
Other                                         5
Green Party                                   4
Plaid Cymru                                   1
Name: generalElectionVoteRetro2017W18, dtype: int64

In [216]:
search(BES_Panel,"gen.*Retro")

generalElectionVoteRetro2017W18     8063
econGenRetroW1                     30549
econGenRetroW2                     30260
econGenRetroW3                     27829
econGenRetroW4                     31550
genElecTurnoutRetroW6              30258
econGenRetroW6                      7522
econGenRetroW7                     30895
econGenRetroW8                     15200
econGenRetroW10                    30237
econGenRetroW11                    30956
econGenRetroW12                     8984
genElecTurnoutRetroW13             31136
econGenRetroW13                    31136
econGenRetroW14                    31063
econGenRetroW15                    30842
econGenRetroW16                    37959
econGenRetroW17                    34366
genElecTurnoutRetro2017W18          9369
genElecTurnoutRetroW19             32177
dtype: int64

In [220]:
BES_Panel["generalElectionVoteRetro2017W18"].value_counts()

Conservative                                3478
Labour                                      2409
Liberal Democrat                             793
United Kingdom Independence Party (UKIP)     416
Scottish National Party (SNP)                336
Don't know                                   236
Green Party                                  224
Other                                        119
Plaid Cymru                                   52
Name: generalElectionVoteRetro2017W18, dtype: int64

In [217]:
BES_Panel["genElecTurnoutRetro2017W18"].value_counts()

Yes, voted    8177
2.0            962
Don't know     230
Name: genElecTurnoutRetro2017W18, dtype: int64

In [222]:
BES_Panel["genElecTurnoutRetro2017W18"][BES_Panel["voted2017W19"]=="No, did not vote"].value_counts()

2.0           465
Yes, voted    204
Don't know     44
Name: genElecTurnoutRetro2017W18, dtype: int64

In [223]:
BES_Panel["generalElectionVoteRetro2017W18"][BES_Panel["voted2017W19"]=="No, did not vote"].value_counts()

Conservative                                73
Labour                                      46
United Kingdom Independence Party (UKIP)    27
Liberal Democrat                            26
Scottish National Party (SNP)               11
Don't know                                   7
Green Party                                  4
Other                                        3
Plaid Cymru                                  0
Name: generalElectionVoteRetro2017W18, dtype: int64

In [ ]:
BES_Panel["genElecTurnoutRetro2017W18"].value_counts()

In [186]:
BES_Panel["generalElectionVoteRetro2017W18"].value_counts()

Conservative                                3478
Labour                                      2409
Liberal Democrat                             793
United Kingdom Independence Party (UKIP)     416
Scottish National Party (SNP)                336
Don't know                                   236
Green Party                                  224
Other                                        119
Plaid Cymru                                   52
Name: generalElectionVoteRetro2017W18, dtype: int64

In [165]:
BES_Panel["generalElectionVoteW19"][last_chunk_unassigned].astype('object').replace(replace_dict)\
        .astype('category').cat.set_categories(pan_dataset_votes[new_vote_var+"_complete"].cat.categories,ordered =True)

37                              Labour
57                              Labour
87                        Conservative
90                          Don't know
91       Scottish National Party (SNP)
                     ...              
31960                           Labour
32031                            Green
32080                           Labour
32155                           Labour
32158                     Conservative
Name: generalElectionVoteW19, Length: 734, dtype: category
Categories (11, object): [Conservative < Labour < Liberal Democrat < Scottish National Party (SNP) ... Other < Don't know < Did not vote < Not Present]

In [66]:
# %%time
# pattern = "profile_eurefturnout_Yes"
# name = "profile_eurefturnout_Yes"
# corr_type = "spearman"
# mask = BES_Panel["profile_eurefturnout"].notnull()
# corr_df,corr_type = make_corr_summary(BES_reduced_with_na[mask].astype('float32'),
#                                       name, corr_type, pattern=pattern)
# display_corr(corr_df, name, corr_type, text_wrap_length=50)

In [75]:
BES_Panel["pastvote_ge_2019W19"].value_counts().sum(), BES_Panel["wave19"].sum()

(31443, 32806)

In [141]:

pan_dataset_votes.loc[last_chunk_unassigned,new_vote_var+"_complete"].cat.categories
# = BES_Panel["generalElectionVoteW19"][last_chunk_unassigned].astype('object').replace(replace_dict).values

Index(['Conservative', 'Labour', 'Liberal Democrat',
       'Scottish National Party (SNP)', 'Plaid Cymru', 'Brexit Party', 'Green',
       'Other', 'Don't know', 'Did not vote', 'Not Present'],
      dtype='object')

In [140]:
BES_Panel["generalElectionVoteW19"][last_chunk_unassigned].astype('object').replace(replace_dict).value_counts(dropna=False)

Conservative                                224
Labour                                      209
Don't know                                   86
Liberal Democrat                             69
Other                                        50
Green                                        36
Scottish National Party (SNP)                32
Brexit Party                                 24
United Kingdom Independence Party (UKIP)      3
Plaid Cymru                                   1
Name: generalElectionVoteW19, dtype: int64

In [142]:
BES_Panel["pastvote_ge_2019W19"].value_counts()

Conservative                     12666
Labour                            8244
Liberal Democrat                  3579
Did not vote                      3159
Scottish National Party (SNP)     1249
Green                              858
Brexit Party                       722
Other                              553
Don't know                         231
Plaid Cymru                        182
Name: pastvote_ge_2019W19, dtype: int64

In [ ]:
[BES_Panel["pastvote_ge_2019W19"]=="Other"]

In [78]:
GE2019_unaccounted = (BES_Panel["wave19"] & BES_Panel["pastvote_ge_2019W19"].isnull())
# 1363 unaccounted for

In [80]:
BES_Panel[GE2019_unaccounted]["profile_turnout_2017"].value_counts()

Yes, voted          608
No, did not vote    107
Don't know           16
Name: profile_turnout_2017, dtype: int64

In [85]:
# search(BES_Panel,"wt_new")
nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])

# bunch of people who only appear to exist in wave/id/wt_full_W1/gor/country/pcon/pano variables!
# 629

In [103]:
# BES_Panel[nowtsW19].notnull().sum().sort_values().tail(60)
# nowtsW19.sum()

In [106]:
BES_Panel["pastvote_ge_2019W19"].value_counts().sum() - BES_Panel["wt_new_W19_result"].notnull().sum()

-734

In [109]:
last_chunk_unassigned = (BES_Panel["pastvote_ge_2019W19"].isnull() & BES_Panel["wt_new_W19_result"].notnull())

In [111]:
BES_Panel[last_chunk_unassigned]["profile_turnout_2017"].value_counts()

Yes, voted          608
No, did not vote    107
Don't know           16
Name: profile_turnout_2017, dtype: int64

In [128]:
BES_Panel[last_chunk_unassigned]["generalElectionVoteW19"].value_counts()

Conservative                                224
Labour                                      209
Don't know                                   86
Liberal Democrat                             69
Other                                        45
Green Party                                  36
Scottish National Party (SNP)                32
Brexit Party                                 24
An independent candidate                      5
United Kingdom Independence Party (UKIP)      3
Plaid Cymru                                   1
Name: generalElectionVoteW19, dtype: int64

In [127]:
BES_Panel["pastvote_ge_2019W19"].cat.categories

Index(['Conservative', 'Labour', 'Liberal Democrat',
       'Scottish National Party (SNP)', 'Plaid Cymru', 'Brexit Party', 'Green',
       'Other', 'Don't know', 'Did not vote'],
      dtype='object')

In [ ]:
replace_dict = {'Green Party':'Green','An independent candidate':'Other'}

In [117]:
BES_Panel[last_chunk_unassigned]["generalElectionVoteW19"].value_counts().sum()

734

In [119]:
BES_Panel[last_chunk_unassigned]["generalElectionVoteW18"].value_counts().sum()

482

In [121]:
BES_Panel[last_chunk_unassigned]["generalElectionVoteW17"].value_counts().sum()

453

In [493]:
LD2005 = pan_dataset_votes["2005GEVote_complete"]=="Liberal Democrat"
LD2010 = pan_dataset_votes["2010GEVote_complete"]=="Liberal Democrat"
LD05_10 = LD2005|LD2010
LD05_10.sum()

16055

In [499]:
LD05_10_tot = (pan_dataset_votes["2005GEVote_complete"]!="Unknown")|(pan_dataset_votes["2010GEVote_complete"]!="Unknown")

In [502]:
LAB2015 = pan_dataset_votes["2015GEVote_complete"]=="Labour"
LAB2017 = pan_dataset_votes["2017GEVote_complete"]=="Labour"
LAB2019 = pan_dataset_votes["2019GEVote_complete"]=="Labour"


In [501]:
(LAB2015&LD05_10).sum()/(LAB2015&LD05_10_tot).sum()

0.11851465117796776

In [503]:
(LAB2015&LD05_10).sum()/(LAB2015&LD05_10_tot).sum()

0.35422623813336385

In [505]:
(LAB2017&LD05_10).sum()/(LAB2017&LD05_10_tot).sum()

0.31912166304408407

In [506]:
(LAB2019&LD05_10).sum()/(LAB2019&LD05_10_tot).sum()

0.3342492322611928